In [1]:
# importing the required packages and functions
import PyPDF2, numpy as np, pandas as pd # NOTE: Use the version 2.12.1 of PyPDF2 as the newer versions have deprecated many functionalities
from PyPDF2 import PdfFileReader
import re
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pprint import pprint

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
pwd

'C:\\Users\\rsinha\\Desktop\\Comment Tool\\OneDrive_1_2-10-2023'

In [4]:
my_dir = 'C:\\Users\\rsinha\\Desktop\\Comment Tool\\OneDrive_1_2-10-2023\\'

In [5]:
# Creating a "cleaning" function which we'll use to clean comments
def cleaner(x):
    x = str(x)
    x = re.sub(r'\[[0-9]*\]',' ',x)
    x = re.sub(r'\s+',' ',x)
    x = x.lower()
    x = re.sub(r'\d',' ',x)
    x = re.sub(r'\s+',' ',x)
    return x

In [6]:
# Creating a function to convert the contents of a pdf file into a list
def pdf_to_text(pdf):
    f = open(pdf,mode='rb')
    pdf_text = []

    pdf_reader3 = PyPDF2.PdfFileReader(f)

    for p in range(pdf_reader3.numPages):
        page = pdf_reader3.getPage(p)
        pdf_text.append(page.extractText())

    f.close()
    all_text = ' '.join(pdf_text).replace('\n','')
    #cleaned_text = cleaner(all_text).split(' re: ')[-1]
    
    return all_text

In [7]:
import os

In [8]:
os.listdir(my_dir)

['.ipynb_checkpoints',
 'api.py',
 'App',
 'App.rar',
 'Chatbot-OpenAPI.ipynb',
 'chatbot_demo_file_source.json',
 'chatbot_demo_file_source_2.json',
 'chatbot_demo_only_comment_and_embedding.json',
 'Duplication-extended-2.ipynb',
 'Duplication-extended.ipynb',
 'Duplications-extended-3.ipynb',
 'EPA-HQ-OW-2017-0300-0939_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1420_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1425_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1431_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1459_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1467_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1475_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1476_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1846_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1850_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1854_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1864_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1867_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1871_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1872_attachment_1.pdf',
 'EPA-HQ-OW-2

In [9]:
pdfs = []
for i in os.listdir(my_dir):
    if '.pdf' in i:
        pdfs.append(i)

In [10]:
pdfs

['EPA-HQ-OW-2017-0300-0939_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1420_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1425_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1431_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1459_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1467_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1475_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1476_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1846_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1850_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1854_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1864_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1867_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1871_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1872_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1882_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1883_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1886_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1887_attachment_1.pdf']

In [11]:
len(pdfs)

19

In [12]:
text_of_pdfs = []
for i in pdfs:
    text_of_pdfs.append(pdf_to_text(i))

Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


In [13]:
len(text_of_pdfs)

19

In [14]:
len(pdfs)

19

In [15]:
#text_of_pdfs

In [16]:
df_comments = pd.DataFrame({'Comment':text_of_pdfs})

In [17]:
df_comments

,Comment
0,"1875 Connecticut Avenue, NW Washington, DC ..."
1,"February 12, 2020 Andrew R. Wheeler Admini..."
2,STATE OF NEVADA Department of Conservation &...
3,"February 12, 2020 Mr. David Ross Assist..."
4,"February 12, 2020 Office of Informat..."
5,"February 11, 2020 Andrew R. Wheeler ..."
6,"February 12, 2020 Mr. David Ross Assis..."
7,~ 1 ~ SPRINGFIELD WATER AND SEWER COMMISSION ...
8,EPN Comments on Pr oposal to Delay Ef fecti v ...
9,...


In [18]:
df_comments['comment_file_name'] = pdfs

In [19]:
df_comments

,Comment,comment_file_name
0,"1875 Connecticut Avenue, NW Washington, DC ...",EPA-HQ-OW-2017-0300-0939_attachment_1.pdf
1,"February 12, 2020 Andrew R. Wheeler Admini...",EPA-HQ-OW-2017-0300-1420_attachment_1.pdf
2,STATE OF NEVADA Department of Conservation &...,EPA-HQ-OW-2017-0300-1425_attachment_1.pdf
3,"February 12, 2020 Mr. David Ross Assist...",EPA-HQ-OW-2017-0300-1431_attachment_1.pdf
4,"February 12, 2020 Office of Informat...",EPA-HQ-OW-2017-0300-1459_attachment_1.pdf
5,"February 11, 2020 Andrew R. Wheeler ...",EPA-HQ-OW-2017-0300-1467_attachment_1.pdf
6,"February 12, 2020 Mr. David Ross Assis...",EPA-HQ-OW-2017-0300-1475_attachment_1.pdf
7,~ 1 ~ SPRINGFIELD WATER AND SEWER COMMISSION ...,EPA-HQ-OW-2017-0300-1476_attachment_1.pdf
8,EPN Comments on Pr oposal to Delay Ef fecti v ...,EPA-HQ-OW-2017-0300-1846_attachment_1.pdf
9,...,EPA-HQ-OW-2017-0300-1850_attachment_1.pdf


## Sentence Similarity

In [20]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2') # using tokenizer from pretrained model
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2') # Assembling the model with pre-trained weights

In [21]:
# Creating an embedding function (basically vectorizing every word in every sentence in order to grab granular details present in each sentence)
def give_embed(sentences):
    def mean_pooling(model_output, attention_mask):

        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    #print("Sentence embeddings:")
    return sentence_embeddings


In [22]:
l1 = df_comments['Comment'].tolist() # maing a list of all the comments

In [23]:
# Ensuring that all the comments belong to string datatype
l2 = []
for i in l1:
    l2.append(str(i))

In [24]:
series_embeds = give_embed(l2) # Applying our embedding function on our corpus

In [25]:
# Evaluating the similarity for each combination of comments using the cosine similarity method 
sent1 = []
sent2 = []
similarity = []

for i in range(len(l2)-1):
    #sent1.append(l2[i])
    for j in range(i+1, len(l2)):
        sent1.append(l2[i])
        sent2.append(l2[j])
        similarity.append(cosine_similarity(series_embeds[i].reshape(1,-1) , series_embeds[j].reshape(1,-1) )[0][0])

In [26]:
len(sent1)

171

In [27]:
len(sent2)

171

In [28]:
len(similarity)

171

In [29]:
# creating the dataframe for all combination of comments along with their similarity scores
df_sentence_similarity = pd.DataFrame({'Docuemnt1':sent1, 'Document2': sent2,'Cosine_Similarity_Score': similarity}) 

In [30]:
df_sentence_similarity

,Docuemnt1,Document2,Cosine_Similarity_Score
0,"1875 Connecticut Avenue, NW Washington, DC ...","February 12, 2020 Andrew R. Wheeler Admini...",0.752627
1,"1875 Connecticut Avenue, NW Washington, DC ...",STATE OF NEVADA Department of Conservation &...,0.697234
2,"1875 Connecticut Avenue, NW Washington, DC ...","February 12, 2020 Mr. David Ross Assist...",0.587532
3,"1875 Connecticut Avenue, NW Washington, DC ...","February 12, 2020 Office of Informat...",0.773077
4,"1875 Connecticut Avenue, NW Washington, DC ...","February 11, 2020 Andrew R. Wheeler ...",0.709509
...,...,...,...
166,State Water Resources Control Board April 1...,"Campaign for Lead Free Water April 12, 202...",0.839006
167,State Water Resources Control Board April 1...,Docket ID No. EPA-HQ-OW-2017-0300 12 April 20...,0.694601
168,"April 12, 2021 Department of Environme...","Campaign for Lead Free Water April 12, 202...",0.794141
169,"April 12, 2021 Department of Environme...",Docket ID No. EPA-HQ-OW-2017-0300 12 April 20...,0.628899


In [31]:
#df_sentence_similarity.to_csv('Prathamesh_sentence_similarity_19_docs.csv')

In [32]:
#df_sentence_similarity.to_excel('Prathamesh_sentence_similarity_19_docs.xlsx',sheet_name='Sheet1',index=False)

In [33]:
df_sentence_similarity[df_sentence_similarity['Cosine_Similarity_Score'] < 0.5]

,Docuemnt1,Document2,Cosine_Similarity_Score
40,STATE OF NEVADA Department of Conservation &...,EPN Comments on Pr oposal to Delay Ef fecti v ...,0.411841
55,"February 12, 2020 Mr. David Ross Assist...",EPN Comments on Pr oposal to Delay Ef fecti v ...,0.411785
94,"February 12, 2020 Mr. David Ross Assis...",EPN Comments on Pr oposal to Delay Ef fecti v ...,0.485759
105,~ 1 ~ SPRINGFIELD WATER AND SEWER COMMISSION ...,EPN Comments on Pr oposal to Delay Ef fecti v ...,0.434768
121,EPN Comments on Pr oposal to Delay Ef fecti v ...,"319 N. Fourth St., Ste. 800 St. Louis, MO 63...",0.493152
123,EPN Comments on Pr oposal to Delay Ef fecti v ...,"April 12, 2021 Department of Environme...",0.411293
124,EPN Comments on Pr oposal to Delay Ef fecti v ...,"Campaign for Lead Free Water April 12, 202...",0.488121


## KEYWORD EXTRACTION
ml6team/keyphrase-extraction-distilbert-inspec

In [34]:
from transformers import (
    TokenClassificationPipeline,
    AutoModelForTokenClassification,
    AutoTokenizer,
)
from transformers.pipelines import AggregationStrategy
import numpy as np

# Define keyphrase extraction pipeline
class KeyphraseExtractionPipeline(TokenClassificationPipeline):
    def __init__(self, model, *args, **kwargs):
        super().__init__(
            model=AutoModelForTokenClassification.from_pretrained(model),
            tokenizer=AutoTokenizer.from_pretrained(model),
            *args,
            **kwargs
        )

    #def postprocess(self, model_outputs):
     #   results = super().postprocess(
     #       model_outputs=model_outputs,
     #       aggregation_strategy=AggregationStrategy.FIRST,
      #  )
       # return np.unique([result.get("word").strip() for result in results])
    def postprocess(self, all_outputs):
        results = super().postprocess(
            all_outputs=all_outputs,
            aggregation_strategy=AggregationStrategy.FIRST,
        )
        return np.unique([result.get("word").strip() for result in results])

In [35]:
# Importing the model and extractor from huggingface.co
model_name = "ml6team/keyphrase-extraction-distilbert-inspec"
extractor = KeyphraseExtractionPipeline(model=model_name)

Downloading:   0%|          | 0.00/697 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/266M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [36]:
# Creating a keyword extracting function
def key_word_extractor(x):
    #try:
    if len(x) > 0:
        return extractor(x)
    else:
        return np.nan
    #except:
    #    return np.nan

In [37]:
# Importing the model and extractor from huggingface.co
model_name = "ml6team/keyphrase-extraction-distilbert-inspec"
extractor = KeyphraseExtractionPipeline(model=model_name)

In [38]:
# Creating a keyword extracting function
def key_word_extractor(x):
    #try:
    if len(x) > 0:
        return extractor(x)
    else:
        return np.nan
    #except:
     #   return np.nan

In [39]:
key_word_extractor('Hello I did my studies in Machine Learning.')

array(['machine learning'], dtype='<U16')

'1875 Connecticut Avenue, NW  Washington, DC 20009  T 202 387 3500  F 202 234 6049  edf.org New York, NY / Austin, TX / Bentonville, AR / Boston, MA / Boulder, CO / Raleigh, NC   Sacramento, CA / San Francisco, CA / Washington, DC / Beijing, China / La Paz, Mexico  Totally chlorine free 100% post -consumer recycled paper   December 1 2, 2019   Mr. David Ross  Assistant Administrator  Office of Water  U.S. Environmental Protection Agency  1200 Pennsylvania Ave. NW  Mail code: 4101M  Washington, DC 20460 –0001   RE:  Comments regarding the lead service line definition and inventory aspects of the agency’s proposed revisions to its Lead and Copper Rule in the National Primary Drinking Water Regulations, Docket No. EPA –HQ–OW-2017 -0300   Dear Assistant Administrator Ross:   Environmental Defense Fund (EDF) respectfully submits the followi ng comments regarding the lead service line (LSL) definition and inventory aspects of the Environmental Protection Agency’s (EPA) proposed revisions to 

In [40]:
df_comments['keywords'] = df_comments['Comment'].apply(lambda x: key_word_extractor(x)) # Extracting out the keywords from each comment

In [ ]:
df_comments

In [34]:
# Creating a list which contains the number of keywords of each comment
key_cnt = []
for i in range(len(df_comments)):
    try:
        key_cnt.append(len(df_comments['keywords'][i]))
    except:
        key_cnt.append(0)

In [35]:
#key_cnt

In [36]:
df_comments['key_count'] = key_cnt # Creating a new column which contains the number of keywords in each comment

In [37]:
df_comments

,Comment,keywords,key_count
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18


In [38]:
rev_len = [len(x.split()) if x is not np.nan else 0 for x in df_comments['Comment']]  # Creating a list which contains the number of words in each comment

In [39]:
#rev_len

In [40]:
df_comments['review_length'] = rev_len # Creating a new column which contains the number of words in each comment

In [41]:
df_comments

,Comment,keywords,key_count,review_length
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409


In [42]:
df_comments['proportion_keywords'] = 100 * df_comments['key_count']/df_comments['review_length'] # proportion of key words present in each comment depending on the context of each comment

In [43]:
df_comments

,Comment,keywords,key_count,review_length,proportion_keywords
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014


# SENTIMENT ANALYSIS

In [44]:
sia = SentimentIntensityAnalyzer() # Creating an instance of SentimentIntensityAnalyzer function to analyze the sentiment of each comment by calling its polarity_scores method

In [45]:
# Creating a function to bin a compound polarity score into 9 categories namely: Extremely Negative, Very Negative, Negative, Neutral-Negative, Neutral, Neutral-Positive, Positive, Very Positive, Extremely Positive
def final_sent_calc(x):
    
    def estimator(a):
        return float(str(a)[:6])
    def estimator2(b):
        return float(str(b)[:7])

    if x in np.asarray(pd.Series(np.arange(-1.001, -0.778,0.0001)).apply(lambda x: estimator2(x))):
        return 'Extremely Negative'
    elif x in np.asarray(pd.Series(np.arange(-0.778, -0.556,0.0001)).apply(lambda x: estimator2(x))):
        return 'Very Negative'
    elif x in np.asarray(pd.Series(np.arange(-0.556, -0.333,0.0001)).apply(lambda x: estimator2(x))):
        return 'Negative'
    elif x in np.asarray(pd.Series(np.arange(-0.333, -0.111,0.0001)).apply(lambda x: estimator2(x))):
        return 'Neutral-Negative'
    elif x in np.asarray(pd.Series(np.arange(-0.111, 0.000,0.0001)).apply(lambda x: estimator2(x))):
        return 'Neutral'
    elif x in np.asarray(pd.Series(np.arange(0, 0.111,0.0001)).apply(lambda x: estimator(x))):
        return 'Neutral'
    elif x in np.asarray(pd.Series(np.arange(0.111, 0.333,0.0001)).apply(lambda x: estimator(x))):
        return 'Neutral-Positive'
    elif x in np.asarray(pd.Series(np.arange(0.333, 0.556,0.0001)).apply(lambda x: estimator(x))):
        return 'Positive'
    elif x in np.asarray(pd.Series(np.arange(0.556, 0.778,0.0001)).apply(lambda x: estimator(x))):
        return 'Very Positive'
    elif x in np.asarray(pd.Series(np.arange(0.778, 1.001,0.0001)).apply(lambda x: estimator(x))):
        return 'Extremely Positive'

In [46]:
df_comments['sentiment_score'] = df_comments['Comment'].apply(lambda x: sia.polarity_scores(x)['compound']) # Applying polarity score method to get the compound polarity score of each comment

In [47]:
df_comments['sentiment'] = df_comments['sentiment_score'].apply(lambda x: final_sent_calc(x)) # Evaluating the sentiment type by applying the binning function

In [48]:
df_comments

,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive


In [49]:
#df_comments.to_csv('Master-General.csv')

In [50]:
#df_sentence_similarity.to_csv('Master-Similarity.csv')

## Summarization

In [51]:
#from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [52]:
def summary_func(x):
    word_count = int(len(x.split()))
    limit = 500
    if word_count > limit:
        return summarizer(' '.join(x.split()[:limit]))[0]['summary_text']
    else:
        return summarizer(x)[0]['summary_text']

In [53]:
df_comments['summary'] = df_comments['Comment'].apply(lambda x: summary_func(x))

In [54]:
df_comments

,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive,epn comments on pr oposal to delay ef fecti v ...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."


In [55]:
df_comments['summary'][0]

'Environmental defense fund submits comments on lead service line definition and inventory aspects of the environmental protection agency’s (epa) proposed revisions to its lead and copper rule (lcr) The agency published proposed revisions in the november , , federa l register at fed reg. , and is accepting comments at docket no. epa -hq-ow- - . edf’ s mission is to preserve the natural systems on which all life depends.'

In [56]:
df_comments['keywords'][0]

array(['- lead', 'comments', 'connectors', 'copper rule',
       'customers misleading', 'economics', 'environmental defense fund',
       'environmental problems', 'environmental protection agency',
       'goosenecks', 'health', 'health program', 'human health',
       'lead service line', 'lead service line definition',
       'national primary drinking water regulations', 'pigtails',
       'policy experts', 'pollution', 'toxic chemicals', 'water flow',
       'water systems'], dtype='<U43')

In [95]:
df_sentence_similarity

,Docuemnt1,Document2,Cosine_Similarity_Score
0,comments regarding the lead service line defin...,national primary drinking water regulations: p...,0.737591
1,comments regarding the lead service line defin...,state of nevada department of conservation & ...,0.686464
2,comments regarding the lead service line defin...,"february , mr. david ross assistant administr...",0.586471
3,comments regarding the lead service line defin...,national primary drinking water regulations: p...,0.780700
4,comments regarding the lead service line defin...,national primary drinking water regulations: p...,0.736849
...,...,...,...
166,proposed delay of effective and compliance dat...,revisions to lead & copper na onal primary dri...,0.696102
167,proposed delay of effective and compliance dat...,docket id no. epa-hq-ow- - april jeff cohen s...,0.734126
168,proposed delay of effective and compliance dat...,revisions to lead & copper na onal primary dri...,0.584698
169,proposed delay of effective and compliance dat...,docket id no. epa-hq-ow- - april jeff cohen s...,0.596619


In [57]:
#df_comments['Comment'][0]

In [58]:
#pd.DataFrame(df_comments.iloc[4:7,:].to_dict())

In [59]:
#df_comments

In [60]:
def search_keyword(search_term='revisions'):
    search = search_term
    search = search.lower()
    #idxs = []
    df = pd.DataFrame()
   # cnt=0
    for i in range(len(df_comments['keywords'])):
        if search in df_comments['keywords'][i]:
            #cnt += 1
            df = df.append(df_comments.iloc[i,:])
        else:
            continue
   
    return df

In [61]:
search_keyword('copper rule revisions')

,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...
11,national primary drinking water regulations: p...,"[bermuda, children, compliance, control, coppe...",16,2024,0.790514,0.9873,Extremely Positive,The international society for environmental ep...
13,comments on proposed delay of effective and co...,"[compliance, copper rule, copper rule revision...",11,3424,0.321262,0.9986,Extremely Positive,The natural resources defense council (nrdc) s...
15,proposed delay of effective and compliance dat...,[california state water resources control boar...,12,945,1.269841,0.9967,Extremely Positive,California state water resources control board...


In [62]:
search_keyword('compliance')

,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive,epn comments on pr oposal to delay ef fecti v ...
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...
11,national primary drinking water regulations: p...,"[bermuda, children, compliance, control, coppe...",16,2024,0.790514,0.9873,Extremely Positive,The international society for environmental ep...
12,"lcr epa revision of hq-ow - comments april , ...","[compliance, corrosion control, cotruvo phd, l...",11,1346,0.817236,0.9826,Extremely Positive,Lcr epa revision of hq-ow was poorly conc eive...
13,comments on proposed delay of effective and co...,"[compliance, copper rule, copper rule revision...",11,3424,0.321262,0.9986,Extremely Positive,The natural resources defense council (nrdc) s...
15,proposed delay of effective and compliance dat...,[california state water resources control boar...,12,945,1.269841,0.9967,Extremely Positive,California state water resources control board...
16,proposed delay of effective and compliance dat...,"[alaska drinking water program, alaska drinkin...",13,231,5.627706,0.9847,Extremely Positive,The state and territorial primacy agencies are...
18,docket id no. epa-hq-ow- - april jeff cohen s...,"[brass faucets, compliance, contaminant, corro...",14,770,1.818182,0.2006,Neutral-Positive,i worked for the us epa in various programs an...


# use gensim word2vec to search in a better using 'most_similar_word'

In [63]:
all_keywords = []
for i in range(len(df_comments['keywords'])):
    for j in df_comments['keywords'][i]:
        all_keywords.append(j)

In [64]:
all_keywords_str = ' '.join(all_keywords)

In [65]:
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rsinha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [66]:
def my_tokenizer(x):
    x = nltk.sent_tokenize(x)

    sentences = [nltk.word_tokenize(sentence) for sentence in x]

    for i in range(len(sentences)):
        sentences[i] = [word for word in sentences[i] if word not in stopwords.words('english')]
    return sentences

In [67]:
keywords_tokenized = my_tokenizer(all_keywords_str)

In [68]:
#keywords_tokenized

In [69]:
tokenized_str = []
for i in range(len(keywords_tokenized)):
    tokenized_str.append(' '.join(keywords_tokenized[i]))
#tokenized_str

In [70]:
tokenized_str_final = ' '.join(tokenized_str)
#tokenized_str_final

In [71]:
search_model = Word2Vec(keywords_tokenized, min_count=1)

In [72]:
'changes' in tokenized_str_final

False

In [73]:
similar = search_model.wv.most_similar('copper')
similar

[('sewer', 0.26975303888320923),
 ('agency', 0.2586798369884491),
 ('phd', 0.2455655187368393),
 ('laws', 0.22235117852687836),
 ('household', 0.22090166807174683),
 ('conformity', 0.22009442746639252),
 ('tracking', 0.2133825272321701),
 ('docket', 0.2117016315460205),
 ('protective', 0.2098371684551239),
 ('act', 0.20787814259529114)]

In [74]:
search_model.wv.most_similar_cosmul(['revisions','delay'])

[('techniques', 0.3857749104499817),
 ('contamination', 0.3642527759075165),
 ('education', 0.3492853343486786),
 ('voluntary', 0.34888383746147156),
 ('county', 0.34589874744415283),
 ('pigtails', 0.34202420711517334),
 ('rity', 0.3409615755081177),
 ('believ', 0.33576521277427673),
 ('housing', 0.3342360258102417),
 ('laws', 0.33365651965141296)]

In [75]:
def search_finder_advanced(search_term="revisions"):
    search_term = search_term.lower()
    df = pd.DataFrame()
    try:
        similar_terms = []
        for term in search_term.split():
            similar_terms.append(term)
        #similar_terms.append(search_term)
        #for n in similar_terms:

        for a,b in search_model.wv.most_similar(similar_terms):
            similar_terms.append(a.lower())
        for i in range(len(df_comments['keywords'])):
            results = [j for j in similar_terms if j in ' '.join(df_comments['keywords'][i])]
            if len(results) > 0:
                df = df.append(df_comments.iloc[i,:])
        if len(df) == 0:
            print('Test-1')
            for i in search_term.split():
                similar_terms.append(i)
            for i in range(len(df_comments['Comment'])):
                results = [j for j in similar_terms if j in df_comments['Comment'][i]]
                df = df.append(df_comments.iloc[i,:])
        print('Test-2')
        return df
    except:
        print('Test-3')
        search = search_term
        search = search.lower()
        #idxs = []
        df = pd.DataFrame()
       # cnt=0
        for i in range(len(df_comments['Comment'])):
            if search in df_comments['Comment'][i]:
                #cnt += 1
                df = df.append(df_comments.iloc[i,:])
            else:
                continue
        return df

In [76]:
search_finder_advanced('revisions')

Test-2


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...


In [77]:
search_finder_advanced('copper rule revisions')

Test-2


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...


In [78]:
search_finder_advanced('copper')

Test-2


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...


In [79]:
search_finder_advanced('compliance')

Test-2


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive,epn comments on pr oposal to delay ef fecti v ...
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...
11,national primary drinking water regulations: p...,"[bermuda, children, compliance, control, coppe...",16,2024,0.790514,0.9873,Extremely Positive,The international society for environmental ep...
12,"lcr epa revision of hq-ow - comments april , ...","[compliance, corrosion control, cotruvo phd, l...",11,1346,0.817236,0.9826,Extremely Positive,Lcr epa revision of hq-ow was poorly conc eive...


In [80]:
search_finder_advanced('alaska')

Test-2


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive,epn comments on pr oposal to delay ef fecti v ...
12,"lcr epa revision of hq-ow - comments april , ...","[compliance, corrosion control, cotruvo phd, l...",11,1346,0.817236,0.9826,Extremely Positive,Lcr epa revision of hq-ow was poorly conc eive...
14,comment to proposed rulemaking docket id no. e...,"[children, community empowerment, dutch town s...",18,1681,1.070791,0.9833,Extremely Positive,Comments support the proposed delay of the lcr...
16,proposed delay of effective and compliance dat...,"[alaska drinking water program, alaska drinkin...",13,231,5.627706,0.9847,Extremely Positive,The state and territorial primacy agencies are...
17,revisions to lead & copper na onal primary dri...,"[coalibon, corrosion control treatment irrespe...",13,25933,0.050129,0.9960,Extremely Positive,The EPA’s proposed revisions to the lead and c...
18,docket id no. epa-hq-ow- - april jeff cohen s...,"[brass faucets, compliance, contaminant, corro...",14,770,1.818182,0.2006,Neutral-Positive,i worked for the us epa in various programs an...


In [81]:
search_keyword('alaska')

""


In [82]:
search_finder_advanced('changes')

Test-3


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...
12,"lcr epa revision of hq-ow - comments april , ...","[compliance, corrosion control, cotruvo phd, l...",11,1346,0.817236,0.9826,Extremely Positive,Lcr epa revision of hq-ow was poorly conc eive...
13,comments on proposed delay of effective and co...,"[compliance, copper rule, copper rule revision...",11,3424,0.321262,0.9986,Extremely Positive,The natural resources defense council (nrdc) s...
17,revisions to lead & copper na onal primary dri...,"[coalibon, corrosion control treatment irrespe...",13,25933,0.050129,0.9960,Extremely Positive,The EPA’s proposed revisions to the lead and c...


In [83]:
search_finder_advanced('delay')

Test-2


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive,epn comments on pr oposal to delay ef fecti v ...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...


In [84]:
search_finder_advanced("dutch")

Test-2


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive,epn comments on pr oposal to delay ef fecti v ...
11,national primary drinking water regulations: p...,"[bermuda, children, compliance, control, coppe...",16,2024,0.790514,0.9873,Extremely Positive,The international society for environmental ep...


In [85]:
df_comments

,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive,epn comments on pr oposal to delay ef fecti v ...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."


In [86]:
pwd

'C:\\Users\\rsinha\\Desktop\\Comment Tool\\OneDrive_1_2-10-2023'

In [87]:
#df_comments.sort_values('key_count',ascending=False)

In [88]:
#df_comments.to_csv('Master-General-3.csv',index=False)

In [91]:
import jsonify

In [96]:
#df_comments.to_dict()

In [99]:
len(df_comments['summary'][0].split())

70

In [100]:
avg_len =   [ len(i.split()) for i in df_comments['summary'].to_list()]

In [103]:
np.median(avg_len)

58.0

In [105]:
df_sentence_similarity

,Docuemnt1,Document2,Cosine_Similarity_Score
0,comments regarding the lead service line defin...,national primary drinking water regulations: p...,0.737591
1,comments regarding the lead service line defin...,state of nevada department of conservation & ...,0.686464
2,comments regarding the lead service line defin...,"february , mr. david ross assistant administr...",0.586471
3,comments regarding the lead service line defin...,national primary drinking water regulations: p...,0.780700
4,comments regarding the lead service line defin...,national primary drinking water regulations: p...,0.736849
...,...,...,...
166,proposed delay of effective and compliance dat...,revisions to lead & copper na onal primary dri...,0.696102
167,proposed delay of effective and compliance dat...,docket id no. epa-hq-ow- - april jeff cohen s...,0.734126
168,proposed delay of effective and compliance dat...,revisions to lead & copper na onal primary dri...,0.584698
169,proposed delay of effective and compliance dat...,docket id no. epa-hq-ow- - april jeff cohen s...,0.596619


In [107]:
df_comments['summary_length'] = df_comments['summary'].apply(lambda x: len(x.split()))

In [117]:
df_comments['summary_length'].median()

58.0

In [126]:
df_comments[df_comments['summary_length']==df_comments['summary_length'].max()]['Comment'][5]

'national primary drinking water regulations: proposed lead and copper rule revisions, docket no. epa –hq–ow- - dear administrator wheeler : the district of columbia water and sewer authority (dc water) appreciates the u.s. environmental protection agency’s efforts to revise the lead and copper rule. removing lead service lines is a tremendous effort and water systems cannot do so alone. dc water believes epa has taken important steps to put forward a revised rule framework that addresses many stakeholder concerns. we provided detailed comments on several rule provisi ons that would be challenging for water systems and regulators to implement. we hope that these comments will help epa finalize the rule expeditiously. sincerely, david l. gadis ceo & general manager epa proposed lcr revis ions - docket id no. epa -hq-ow- - district of columbia water and sewer authority comments page corrosion control epa described optimization and re -optimization studies in . that have minimal differenc